ex2, create a relu nn

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = '/home/ramon/mount/handwriting/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
batch_size = 128

#the idea for this NN is to have one hidden layer, and use relu to non-linear (wx+b) -> relu -> (wx+b)
#loss function is to calculate the loss at each iterration. optimizer is to minimize the loss
hidden_layer_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables. add one weights for hidden layer
  weights_1 = tf.Variable(
                 tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases_1  = tf.Variable(tf.zeros([hidden_layer_nodes]))

  weights_2 = tf.Variable(
                 tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases_2  = tf.Variable(tf.zeros([num_labels]))
  alpha_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  layer_1 = tf.nn.relu(alpha_1)
  output_layer = tf.matmul(layer_1, weights_2) + biases_2
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(output_layer, tf_train_labels))

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(output_layer)
  v_layer_1 = tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1))
  v_prediction = tf.nn.softmax(tf.matmul(v_layer_1, weights_2) + biases_2)
  t_layer_1 = tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1))
  t_prediction = tf.nn.softmax(tf.matmul(t_layer_1, weights_2) + biases_2)

In [6]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]    
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run( \
                              [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(  \
                     v_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(t_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 384.280518
Minibatch accuracy: 8.6%
Validation accuracy: 27.9%
Minibatch loss at step 500: 5.259560
Minibatch accuracy: 85.2%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 8.136488
Minibatch accuracy: 82.0%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 7.490948
Minibatch accuracy: 82.0%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 12.026291
Minibatch accuracy: 84.4%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 6.916832
Minibatch accuracy: 82.0%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 3.515081
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 88.7%
